<a href="https://colab.research.google.com/github/martinpius/Applied-Predictive-Modeling2/blob/master/Graphs_with_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Go inside tensorflow and keras functionality to understand how things fits together throgh graphics computing

In [6]:
from google.colab import drive
try: 
  drive.mount("/content/drive/", force_remount = True)
  COLAB = True
  import tensorflow as tf
  print(f"You are using Colab with tensorflow version-->{tf.__version__}")
except:
  pass

def timeseteup(x):
  hours = int(x/60*60)
  minutes = int(x % (60 * 60)/60)
  seconds = int(x % 60)
  return f"{hours}:{minutes:>03}: {seconds:>05.2f}"

Mounted at /content/drive/
You are using Colab with tensorflow version-->2.3.0


In [7]:
import timeit
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
plt.style.use("fivethirtyeight")

In [8]:
#Graphs are very useful because they allows tensorflow to run fast and in parallel on multiple devices independently 
#of python interpreter.


In [9]:
#To create graph, we can apply tf.function() either as decorator or directly.
def fn_take_tensors(x,y,k): #Define any function which manipulate tensors
  x = tf.matmul(x,y)
  x = x + k
  return x
myfn = tf.function(fn_take_tensors) #Instatiate the graph 
a = tf.Variable(tf.constant([[2,1],[4,2]], dtype = tf.float32))
b = tf.Variable(tf.constant([[1,2],[3,1]], dtype = tf.float32))
c = tf.Variable(3.)
graph1 = myfn(a, b, c).numpy()#carries out computation


In [10]:
graph1

array([[ 8.,  8.],
       [13., 13.]], dtype=float32)

In [11]:
#tf.function as  decorator
def inner_fn(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x
@tf.function()
def outer_fn(x):
  y = tf.constant([[2,1],[3,5]], dtype=tf.float32)
  b = tf.constant(4, dtype = tf.float32)
  return inner_fn(x, y, b)

myfn = outer_fn(tf.constant([[2,2],[1,3]], dtype = tf.float32))

In [12]:
myfn

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 16.],
       [15., 20.]], dtype=float32)>

In [13]:
#Flow charts using loops and conditions
def flowchart(x):
  if tf.reduce_sum(x) <=1:
    return x*x
  else:
    return x - 1

myflow = tf.function(flowchart)
print(f"out: {myflow(tf.constant([[0.003,0.002],[-9,0.1]], dtype = tf.float32))}")

out: [[9.0000003e-06 4.0000004e-06]
 [8.1000000e+01 1.0000001e-02]]


In [14]:
print(tf.autograph.to_code(flowchart)) #Check out how the flowchart  function (Python) is converted into Tensorflow ops

def tf__flowchart(x):
    with ag__.FunctionScope('flowchart', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            (do_return, retval_) = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = (ag__.ld(x) * ag__.ld(x))
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = (ag__.ld(x) - 1)
            except:
                do_return = False
                raise
        ag__.if_stmt((ag__.converted_call(ag__.ld(tf).reduce_sum, (ag__.l

In [15]:
import tensorflow as tf

#Observe how the graph speedup excecution time for complicated computations
class MySpeed(tf.keras.Model):
  def __init__(self, **kwargs):
    super(MySpeed, self).__init__(**kwargs)
    self.flatten = tf.keras.layers.Flatten(input_shape = (128,128))
    self.dense_1 = tf.keras.layers.Dense(units = 128, activation = 'relu')
    self.dense_2 = tf.keras.layers.Dense(units = 64, activation = 'relu')
    self.dropout = tf.keras.layers.Dropout(rate = 0.2)
    self.out = tf.keras.layers.Dense(units = 10, activation = 'softmax')
  
  def call(self, inputs):
    inputs = self.flatten(inputs)
    inputs = self.dense_1(inputs)
    inputs = self.dense_2(inputs)
    return self.out(inputs)

myinput = tf.random.uniform([256,128,128])
fn1 = MySpeed()#Normal
graph2 = tf.function(fn1)#graph

print('time1', timeseteup(timeit.timeit(lambda: fn1(myinput), number = 10000)))
print('time2',timeseteup(timeit.timeit(lambda: graph2(myinput), number = 10000)))

time1 6:000: 06.00
time2 4:000: 04.00


In [45]:
#Shifting to eager excecution by adding a customized eagerly layer in a keras sequential model
class MyEager(tf.keras.layers.Layer):
  def __init__(self,**kwargs):
    super(MyEager, self).__init__(**kwargs)
  
  def call(self, inputs):
    print("\nWe are running Eagerly:", str(datetime.now()))
    return inputs



In [46]:
#Using the above keras layer for eager execution
class NewSequential(tf.keras.Model):
  def __init__(self):
    super(NewSequential, self).__init__()
    self.flatten = tf.keras.layers.Flatten(input_shape = (28,28))
    self.layer1 = tf.keras.layers.Dense(units = 128, activation = 'relu')
    self.dropout = tf.keras.layers.Dropout(rate = 0.2)
    self.layer2 = tf.keras.layers.Dense(units = 64, activation = 'relu')
    self.myeager = MyEager()
    self.out = tf.keras.layers.Dense(units = 10)
  
  def call(self, inputs):
    inputs = self.flatten(inputs)
    inputs = self.layer1(inputs)
    inputs = self.dropout(inputs)
    inputs = self.layer2(inputs)
    inputs = self.out(inputs)
    return self.myeager(inputs)

x = tf.random.uniform([60, 28,28])
eager1 = NewSequential()
y = tf.random.uniform([60])
mylos = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
eager1.compile(run_eagerly = False,loss = mylos)
eager1.fit(x, y, epochs = 10)


Epoch 1/10

We are running Eagerly: 2020-11-18 10:15:59.800193

We are running Eagerly: 2020-11-18 10:15:59.934074
2/2 [==============================] - 0s 3ms/step - loss: 1.5939
Epoch 2/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0095
Epoch 3/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 4/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0038
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0023
Epoch 6/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0014
Epoch 7/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 8/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 6.6668e-04
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 6.1425e-04


In [47]:
eager1.compile(run_eagerly= True, loss = mylos)
eager1.fit(x, y, epochs = 10)

Epoch 1/10

We are running Eagerly: 2020-11-18 10:16:08.779854
1/2 [==============>...............] - ETA: 0s - loss: 5.7432e-04
We are running Eagerly: 2020-11-18 10:16:08.804546
2/2 [==============================] - 0s 8ms/step - loss: 3.0640e-04
Epoch 2/10

We are running Eagerly: 2020-11-18 10:16:08.820789
1/2 [==============>...............] - ETA: 0s - loss: 1.7881e-07
We are running Eagerly: 2020-11-18 10:16:08.835541
2/2 [==============================] - 0s 8ms/step - loss: 1.0292e-06
Epoch 3/10

We are running Eagerly: 2020-11-18 10:16:08.851414
1/2 [==============>...............] - ETA: 0s - loss: 1.6019e-07
We are running Eagerly: 2020-11-18 10:16:08.865885
2/2 [==============================] - 0s 8ms/step - loss: 1.0729e-07
Epoch 4/10

We are running Eagerly: 2020-11-18 10:16:08.881819
1/2 [==============>...............] - ETA: 0s - loss: 5.0664e-07
We are running Eagerly: 2020-11-18 10:16:08.896130
2/2 [==============================] - 0s 8ms/step - loss: 4.2319e-07
